In [2]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
plt.style.use('ggplot')
#plt.style.use('fivethirtyeight')
import scipy as sp
import re

In [3]:
# warning 무시
import warnings
warnings.filterwarnings(action='ignore')

### 데이터 불러오기

In [4]:
# 날짜 데이터 불러올 때 
df=pd.read_excel('./data/raw_data_세부.xlsx',encoding='euc-kr',parse_dates={'날짜':["년","월"]})
df=df.rename(columns={'날짜':'date','분류1':'device','분류2':'shopping_type','서비스분류1':'service','종류1':'type1','종류2':'type2','값':'value'})

# 종합_쇼핑기여 제외 (종합 만 )
df=df[df['shopping_type']=='종합']
#df.head(10)

In [5]:
# 광고 타입 (직접광고, 네트워크 ...) 별 매출
## 서비스 전체매출
data_adtype_sales=df.query('type2!=["PV","UV","일평균방문횟수"] and device=="PC" and type1 == "매출 구분"')
data_adtype_sales=data_adtype_sales.dropna(subset=['value'])        # 'value'컬럼에 NA있을때 제거

# service컬럼에 '뉴스' 또는 '기사박스' 일경우 '뉴스'로 통일. 
## TV,스푼피드,여행,이글루스,자동차,증권정보,코인,허브,이슈트렌드는 모두 '서브'로 통일
data_adtype_sales['service_group']=np.where(data_adtype_sales['service'].isin(['기사박스']),'뉴스',data_adtype_sales['service'] )
data_adtype_sales['service_group']=np.where(data_adtype_sales['service'].isin(['TV','스푼피드','여행','이글루스','자동차','증권정보','코인','허브','이슈트렌드']),'서브',data_adtype_sales['service_group'])
print(data_adtype_sales['service_group'].unique())
data_adtype_sales.head()

['프런트' '검색' '뉴스' '서브']


,date,device,shopping_type,service,type1,type2,value,service_group
5,2018-09-01,PC,종합,프런트,매출 구분,직접광고,354000000.0,프런트
6,2018-09-01,PC,종합,프런트,매출 구분,네트워크,59873778.0,프런트
7,2018-09-01,PC,종합,프런트,매출 구분,쇼핑,464000000.0,프런트
9,2018-09-01,PC,종합,프런트,매출 구분,총매출,877000000.0,프런트
22,2018-09-01,PC,종합,검색,매출 구분,제휴,857000000.0,검색


In [18]:
# 광고구분없이 서비스별(프런트, 검색, 뉴스, 서브) 매출 (groupby sum)
data_adtype_sales_groupsum=pd.DataFrame(data_adtype_sales.groupby(['date','service_group'], as_index=False)['value'].sum()) 

In [19]:
# PC, 서비스별 트래픽(PV,UV) 데이터
data_traffic=df.query('device=="PC" and type1=="트래픽" and (type2=="UV" or type2=="PV") ')[['date', 'device','service','type1','type2','value']]
data_traffic['service_group']=np.where(data_traffic['service'].isin(['기사박스']),'뉴스',data_traffic['service'])
data_traffic['service_group']=np.where(data_traffic['service'].isin(['TV','스푼피드','여행','이글루스','자동차','증권정보','코인','허브','이슈트렌드']),'서브',data_traffic['service_group'])

In [20]:
data_traffic_groupsum=pd.DataFrame(data_traffic.groupby(['date','service_group','type2'],as_index=False)['value'].sum())

In [10]:
sales =data_adtype_sales.groupby(['date','service_group']).sum()
sales_pcts=pd.DataFrame(round((sales/sales.groupby(level=0).sum())*100,2).reset_index()['value'])
data_adtype_sales_groupsum=pd.concat([data_adtype_sales_groupsum, sales_pcts],axis=1)
data_adtype_sales_groupsum.columns=['date','service_group','value','pcts']

In [11]:
data_adtype_sales_groupsum['inc_pcts']=round(((sales.groupby(level=1).pct_change())*100).reset_index()['value'],2)

In [21]:
# 서비스별, 광고종류별 매출
### bar-plot으로 총매출 그리고, line-plot으로 직접광고, 쇼핑, 네트워크, 제휴 각각 그리기
### 프런트, 검색, 뉴스, 서브 각각 다른 subplot 4개로
data_adtype_sales_groupsum2=pd.DataFrame(data_adtype_sales.groupby(['date','service_group','type2'],as_index=False)['value'].sum())

In [22]:
# 프론트 트래픽
data_traffic_front=df.query('device=="PC" and service=="프런트" and type1=="트래픽" and (type2=="UV" or type2=="PV") ')[['date', 'device','service','type1','type2','value']]

In [23]:
#  PC, 트래픽 평균단가
data_traffic_price=df.query('device=="PC" and type1=="트래픽 평균단가"')
data_traffic_price= data_traffic_price.dropna(subset=['value'])

# 서비스 카테고리 4개로 축소 : 프런트, 검색, 뉴스, 서브
data_traffic_price['service_group']=np.where(data_traffic_price['service'].isin(['기사박스']),'뉴스', data_traffic_price['service'])
data_traffic_price['service_group']=np.where(data_traffic_price['service'].isin(['TV','스푼피드','여행','이글루스','자동차','증권정보','코인','허브','이슈트렌드']),'서브',data_traffic_price['service_group'])

# 소수점 
data_traffic_price['value']=round(data_traffic_price['value'],2)

In [24]:
data_traffic_price_groupsum=pd.DataFrame(data_traffic_price.groupby(['date','service_group','type2'])['value'].sum().reset_index())

## Plotly-dash
[참고1] https://dash.plot.ly <br>
[참고2] https://dash.plot.ly/dash-core-components


> ### Part2. Layout

> ### Part3. Basic Dash Callbacks
* 

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    html.H1(children='KPI-Dash'),

    html.Div(children='''
        Dash: zum traffic and sales visualization
    '''),

    dcc.Graph(
        id='example-graph',
        figure={
            'data': [
                {'x': list(data_traffic_groupsum.query('service_group=="프런트"and type2=="PV"')['date']),
                 'y':list(data_traffic_groupsum.query('service_group=="프런트"and type2=="PV"')['value']), 
                 'type': 'line', 
                 'name': '프런트_PV'},
                
                  {'x': list(data_traffic_groupsum.query('service_group=="검색"and type2=="PV"')['date']),
                 'y':list(data_traffic_groupsum.query('service_group=="검색"and type2=="PV"')['value']), 
                 'type': 'line', 
                 'name': '검색_PV'},
                
                {'x': list(data_adtype_sales_groupsum.query('service_group=="프런트"')['date']), 
                 'y': list(data_adtype_sales_groupsum.query('service_group=="프런트"')['value']), 
                 'type': 'bar', 
                 'name': '프런트_매출'},
                
                {'x': list(data_adtype_sales_groupsum.query('service_group=="검색"')['date']), 
                 'y': list(data_adtype_sales_groupsum.query('service_group=="검색"')['value']), 
                 'type': 'bar', 
                 'name': '검색_매출'},
            ],
            'layout': {
                'title': 'PC-서비스별-트래픽-매출'
            }
        }
    )
])

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Debugger PIN: 226-693-756
Debugger PIN: 226-693-756
Debugger PIN: 226-693-756
Debugger PIN: 226-693-756
Debugger PIN: 226-693-756
Debugger PIN: 226-693-756
Debugger PIN: 226-693-756
Debugger PIN: 226-693-756
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
